In [74]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import random
import torch.optim as optim
from google.colab import drive
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
!pip install mido
import mido

drive.mount("/drive")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 KB 5.0 MB/s eta 0:00:00
Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [3]:
import numpy as np
import sys
import os

np.set_printoptions(threshold=sys.maxsize)

#out = np.load("/drive/My Drive/outputs/some.mid_1.npy")
#inp =  np.load("/drive/My Drive/inputs/some.mid_1.npy")

In [4]:
input = []
output = []

input_list = os.listdir("/drive/My Drive/inputs")
output_list = os.listdir("/drive/My Drive/outputs")

for path in input_list:
  try:
    arr = np.load('/drive/My Drive/inputs/' + path)
    #print('/drive/My Drive/inputs/' + path ,arr.shape)
    input.append(arr[:, 12:76])
  except:
    pass
input = np.array(input)
input = input[:,np.newaxis , :]

for path in output_list:
  try:
    arr = np.load('/drive/My Drive/outputs/' + path)
    output.append(arr[:, 12:76])
  except:
    pass
output = np.array(output)
output = output[:, np.newaxis,:]

input = torch.Tensor(input)
output = torch.Tensor(output)

input_dataset = TensorDataset(input)
output_dataset = TensorDataset(output)

#print(input_dataset.tensors)

dataloader = DataLoader(dataset = output_dataset , batch_size= 3)
dataloaderG = DataLoader(input_dataset, batch_size= 3 )





In [6]:


manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)


batchsize = 32
inputsize = 4
smallest_note =16
note_range = 129
epochs = 5
rate = 0.0002
adam_beta = 0.5


device = torch.device("cuda:0" if (torch.cuda.is_available() and 1 > 0) else "cpu")


# Sample Data 4 bars smallest note is 16th note:
# Mary had a little lamb 
#   
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

class Generator(nn.Module):
  def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            nn.ConvTranspose2d( 1, 4 * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(4 * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(4 * 8, 4 * 4, 4, 1, 1, bias=False),
            nn.BatchNorm2d(4 * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d( 4 * 4, 4 * 2, 4, 1, 1, bias=False),
            nn.BatchNorm2d(4 * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d( 4 * 2, 4, 4, 1, 1, bias=False),
            nn.BatchNorm2d(4),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d( 4, 1, 4, 1, 1, bias=False),
            nn.Tanh()
        )
  def forward(self, input):
        return self.main(input)


generator = Generator(1).to(device)
generator.apply(weights_init)

# Print the model
print(generator)


class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(1, 2, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(2, 2 * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(2 * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(2 * 2, 2 * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(2 * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(2 * 4, 2 * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(2 * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(2 * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid(),

        )

    def forward(self, input):
        return self.main(input)

discriminator = Discriminator(1).to(device)

discriminator.apply(weights_init)

# Print the model
print(discriminator)




Random Seed:  999
Generator(
  (main): Sequential(
    (0): ConvTranspose2d(1, 32, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(32, 16, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(16, 8, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1), bias=False)
    (7): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(8, 4, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1), bias=False)
    (10): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(4, 1, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)

In [7]:

criterion = nn.BCELoss()

real_label = 1.
fake_label = 0.

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(discriminator.parameters(), lr=rate, betas=(adam_beta, 0.999))
optimizerG = optim.Adam(generator.parameters(), lr=rate, betas=(adam_beta, 0.999))

In [8]:
img_list = []
G_losses = []
D_losses = []
iters = 0

print("Starting Training Loop...")
# For each epoch
for epoch in range(epochs):
    # For each batch in the dataloader
    for i, dataT in enumerate(zip(dataloader,dataloaderG), 0):
        data, dataG = dataT

        discriminator.zero_grad()

        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), real_label, dtype=torch.float, device=device)

        output = discriminator(real_cpu).view(-1)

        errD_real = criterion(output, label)

        errD_real.backward()
        D_x = output.mean().item()

        input = dataG[0].to(device)

        fake = generator(input)

        label.fill_(fake_label)

        output = discriminator(fake.detach()).view(-1)


        errD_fake = criterion(output, label)

        errD_fake.backward()
        D_G_z1 = output.mean().item()

        errD = errD_real + errD_fake

        optimizerD.step()


        generator.zero_grad()
        label.fill_(real_label)  
        output = discriminator(fake).view(-1)

       
        errG = criterion(output, label)
        
        errG.backward()
        D_G_z2 = output.mean().item()
        
        optimizerG.step()

  
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        G_losses.append(errG.item())
        D_losses.append(errD.item())

        iters += 1

Starting Training Loop...
[0/5][0/400]	Loss_D: 1.5083	Loss_G: 0.7025	D(x): 0.4595	D(G(z)): 0.5173 / 0.4956
[0/5][50/400]	Loss_D: 1.0460	Loss_G: 1.0715	D(x): 0.5642	D(G(z)): 0.3641 / 0.3573
[0/5][100/400]	Loss_D: 1.3179	Loss_G: 0.6074	D(x): 0.6570	D(G(z)): 0.5863 / 0.5450
[0/5][150/400]	Loss_D: 0.8832	Loss_G: 1.2345	D(x): 0.6061	D(G(z)): 0.3071 / 0.2992
[0/5][200/400]	Loss_D: 0.4442	Loss_G: 1.5891	D(x): 0.8510	D(G(z)): 0.2332 / 0.2335
[0/5][250/400]	Loss_D: 0.5875	Loss_G: 2.0593	D(x): 0.8072	D(G(z)): 0.3039 / 0.1341
[0/5][300/400]	Loss_D: 0.3900	Loss_G: 1.8551	D(x): 0.8232	D(G(z)): 0.1724 / 0.1679
[0/5][350/400]	Loss_D: 0.4893	Loss_G: 1.4580	D(x): 0.9441	D(G(z)): 0.3325 / 0.2577
[1/5][0/400]	Loss_D: 0.1304	Loss_G: 3.0566	D(x): 0.9298	D(G(z)): 0.0550 / 0.0486
[1/5][50/400]	Loss_D: 0.1490	Loss_G: 2.7656	D(x): 0.9167	D(G(z)): 0.0600 / 0.0630
[1/5][100/400]	Loss_D: 0.0915	Loss_G: 3.3797	D(x): 0.9524	D(G(z)): 0.0415 / 0.0359
[1/5][150/400]	Loss_D: 0.1327	Loss_G: 4.0526	D(x): 0.8934	D(G(z)): 

In [124]:
def enlarge_matrix(matrix):
    
    new_matrix = np.empty((0,matrix.shape[1]), matrix.dtype)
    for i  in range(matrix.shape[1]):
        # Check if the current row index is divisible by 120
        for j in range(120):
            new_matrix = np.vstack((new_matrix, matrix[:,i]))
    return new_matrix

In [139]:
def arry2mid(ary, tempo=500000):
    # get the difference

    new_ary = ary #np.concatenate([np.array([[0] * 71]), np.array(ary)], axis=0)
    changes = new_ary[1:] - new_ary[:-1]
    # create a midi file with an empty track
    mid_new = mido.MidiFile()
    track = mido.MidiTrack()
    mid_new.tracks.append(track)
    track.append(mido.MetaMessage('set_tempo', tempo=tempo, time=0))
    # add difference in the empty track
    last_time = 0
    for ch in changes:
        if set(ch) == {0}:  # no change
            last_time += 1
        else:
            on_notes = np.where(ch > 0)[0]

            on_notes_vol = ch[on_notes]
            off_notes = np.where(ch < 0)[0]
            first_ = True
            for n, v in zip(on_notes, on_notes_vol):

                new_time = last_time if first_ else 0
                track.append(mido.Message('note_on', note=n + 21, velocity=int(v), time=new_time))
                first_ = False
            for n in off_notes:
                new_time = last_time if first_ else 0
                track.append(mido.Message('note_off', note=n + 21, velocity=0, time=new_time))
                first_ = False
            last_time = 0
    return mid_new

In [ ]:

result_array = generator(input[0:1]).cpu().detach().numpy()


#result_array = enlarge_matrix(result_array).transpose()
result_array = np.where(result_array>0.2, 1, 0)
#print(result_array)
result_array = result_array[0,0,:,:]
print(result_array.shape)
plt.plot(range(result_array.shape[0]), np.multiply(np.where(result_array>0, 1, 0), range(1, 72)), marker='.', markersize=1, linestyle='')
plt.title("midi plot")
plt.show()

result_array = enlarge_matrix(result_array)
print(result_array.shape)
mid = arry2mid(result_array*120)

mid.save("test.mid")

In [142]:
validation = input[0].cpu().detach().numpy()[0,:,:]
validation = enlarge_matrix(validation)

mid2 = arry2mid(validation)
mid2.save("validation_input.mid")




'\nout = output[0].cpu().detach().numpy()\nprint(out.shape)\nout = enlarge_matrix(out)\nmid3 = arry2mid(out)\nmid3.save("vslidation_output.mid")\n'